# Sesión 1 - Apartado 5: Taller completo

En este taller aplicaremos todo lo aprendido hasta ahora:

- Limpieza de datos.
- Preprocesamiento de variables numéricas y categóricas.
- Ingeniería de características.
- Construcción de un pipeline completo.
- Evaluación inicial del modelo.


## 1. Carga de datos

In [ ]:
import pandas as pd
import numpy as np

# Dataset
df = pd.read_csv('data/mini_titanic.csv')
df.head()

## 2. Preparación del dataset
- Eliminamos columnas poco útiles (ej: PassengerId).
- Separamos variables predictoras y target.

In [ ]:
X = df.drop(columns=['Survived','PassengerId'])
y = df['Survived']
X.head()

## 3. Ingeniería de características personalizada

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_ = X.copy()
        # Feature: longitud del nombre
        X_['Name_length'] = X_['Name'].apply(len)
        # Feature: inicial del nombre
        X_['Name_initial'] = X_['Name'].str[0]
        # Feature: interacción Edad * Tarifa
        X_['Age_Fare'] = X_['Age'] * X_['Fare']
        return X_

## 4. Construcción del pipeline
Incluiremos:
- Imputación de nulos.
- Escalado de numéricas.
- Codificación de categóricas.
- Ingeniería de características.
- Modelo final.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Columnas por tipo
num_cols = ['Age','Fare']
cat_cols = ['Sex','Embarked','Name_initial']

# Preprocesamiento numérico
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocesamiento categórico
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

# Pipeline completo
pipe = Pipeline(steps=[
    ('features', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

## 5. Evaluación con conjunto de prueba

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)
print("Score en test:", pipe.score(X_test, y_test))

## 6. Ejercicio final
👉 Extiende este pipeline probando:
- Otro modelo (ej: RandomForestClassifier).
- Nuevas features (ej: binning de edad).
- Métricas adicionales (ej: f1-score).